In [1]:
#simply start to run all the cells from here, after you replace the paths bellow

#but you should stop running when you encounter the cell that prints  the message:
#"Now you can stop running, the following is only for comparing to the ground-truth on my local machine."



In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import queue
from collections import deque
import os
import rtree
from rtree import index

In [3]:
path_regular_task = "C:\\Users\\Alienware\\source\\507_Oanea_Smit\\testImages\\regular_tasks"#this must be changed
path_processed_images = "croppedImagesRest"#this is a relative path, don't have to change it
path_initial_image = "_start_train_01.jpg"#this is relative, doesn't need to be changed
path_general_txt_files = "C:\\Users\\Alienware\\source\\507_Oanea_Smit\\testImages\\regular_tasks"#the folder that contains the .txt files, probably the same as the images file

In [4]:
import os
import cv2
import numpy as np

In [5]:
# Define your color list
colors_task = [(160, 101, 83), (160, 94, 72), (152, 83, 54), (174, 104, 79), (161, 85, 59), (134, 73, 55), (132, 69, 52), (152, 76, 50), (142, 77, 55), (128, 67, 46), (125, 67, 47), (124, 65, 47), (128, 70, 48), (119, 62, 45), (119, 50, 35), (134, 62, 47), (148, 88, 87), (155, 92, 83), (233, 192, 208), (162, 96, 74)]
colors_task_part2 = [(165, 101, 92), (167, 108, 90), (166, 107, 103), (171, 106, 88), (182, 111, 91), (165, 81, 45), (172, 89, 59), (186, 102, 76), (154, 80, 67), (129, 63, 47), (125, 68, 49), (144, 83, 64), (125, 66, 50), (126, 67, 49), (100, 54, 41), (118, 66, 53), (93, 43, 34), (106, 54, 40), (104, 56, 46), (75, 28, 20)]
colors_task_part3 = [(169, 96, 79), (150, 75, 56), (115, 49, 33), (125, 53, 41), (135, 66, 51), (127, 51, 35), (116, 43, 28), (107, 44, 27), (127, 55, 40), (106, 48, 36), (100, 40, 29), (117, 50, 31), (113, 49, 37), (121, 60, 41), (127, 61, 39), (85, 32, 16), (118, 57, 36), (115, 53, 38), (138, 71, 52), (124, 59, 39)]
colors_task_part4 = [(94, 32, 21), (100, 38, 27), (94, 32, 19), (111, 42, 27), (106, 44, 29), (120, 54, 32), (118, 50, 27), (134, 60, 33), (123, 53, 28), (112, 55, 38), (115, 56, 42), (116, 57, 39), (100, 44, 29), (95, 38, 21), (118, 56, 41), (112, 49, 31), (123, 57, 33), (126, 67, 51), (117, 55, 34), (130, 72, 50)]

colors_brown_margins = colors_task + colors_task_part2 + colors_task_part3 + colors_task_part4

print("len colors_task = ", len(colors_brown_margins))

len colors_task =  80


In [6]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    pts = np.array(pts)
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

In [7]:
def compute_filtered_image(image, colors_task):
    # Initialize the color_mask with the same size as the original image and a single channel
    height, width, _ = image.shape
    color_mask = np.zeros((height, width), dtype="uint8")

    # Iterate through the colors_task list and create a mask for each color
    for col in colors_task:
        color = (col[2], col[1], col[0])

        lower = np.array([max(c - 2, 0) for c in color], dtype="uint8")
        upper = np.array([min(c + 2, 255) for c in color], dtype="uint8")
        color_filter = cv2.inRange(image, lower, upper)
        color_mask = cv2.add(color_mask, color_filter)

    # Apply the mask to the image
    filtered_image = cv2.bitwise_and(image, image, mask=color_mask)
    
    return color_mask
    #return filtered_image#sau return color mask?

In [8]:
def find_white_pixels(image):
    # Convert the image to grayscale if it is in color
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image.copy()

    # Threshold the grayscale image to obtain a binary image with black and white pixels
    _, binary = cv2.threshold(gray_image, 254, 255, cv2.THRESH_BINARY)

    # Find the indices of the white pixels in the binary image
    white_pixel_indices = np.where(binary == 255)

    # Combine the x and y coordinates of the white pixels into a list of tuples
    white_pixel_coords = list(zip(white_pixel_indices[1], white_pixel_indices[0]))

    return white_pixel_coords

In [9]:
def four_point_transform_whole_table(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    height, width, _ = image.shape

    new_points = np.float32([
        [min(p[0] for p in rect), min(p[1] for p in rect)],
        [max(p[0] for p in rect), min(p[1] for p in rect)],
        [max(p[0] for p in rect), max(p[1] for p in rect)],
        [min(p[0] for p in rect), max(p[1] for p in rect)]
    ])

    M = cv2.getPerspectiveTransform(rect, new_points)
    warped = cv2.warpPerspective(image, M, (width, height))

    # Calculate the minimum and maximum coordinates of the new points with a margin of 5 pixels
    x_min, y_min = (new_points.min(axis=0) - 5).astype(int)
    x_max, y_max = (new_points.max(axis=0) + 5).astype(int)

    # Crop the region of interest and resize it to be 1500*1500 pixels before returning it
    cropped_image = warped[y_min:y_max, x_min:x_max]
    resized_image = cv2.resize(cropped_image, (1500, 1500))

    return resized_image

In [10]:
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def filter_points_by_distance(points, min_distance=7, min_neighbors=2):
    filtered_points = []
    
    for i, p1 in enumerate(points):
        if i%100 == 0:
            print("i = ", i)
        close_neighbors = 0
        for p2 in points[i + 1:]:
            if euclidean_distance(p1, p2) < min_distance:
                close_neighbors += 1

            if close_neighbors >= min_neighbors:
                filtered_points.append(p1)
                break
                
    return filtered_points

def filter_points_by_distance_rtree(points, min_distance):
    # Initialize an R-Tree index
    idx = index.Index()

    # Populate the R-Tree index with point data
    for i, point in enumerate(points):
        idx.insert(i, point + point)  # Duplicate the point coordinates for bounding box

    filtered_points = []
    ind = 0
    for point in points:
        if ind%20000==0:
            print("\tfiltering = ", ind)
        ind+=1
        nearby_points = list(idx.intersection((point[0] - min_distance, point[1] - min_distance, point[0] + min_distance, point[1] + min_distance)))
        if len(nearby_points) > 5:  # The point itself is included in the result
            filtered_points.append(point)
    
    return filtered_points

In [11]:
def profesionally_crop_image(image):
    
    image_original = image.copy()
    color_mask = compute_filtered_image(image, colors_brown_margins)
    #cv2.imshow("ColorMask", color_mask)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    points = find_white_pixels(color_mask)
    print("filtering...")
    print("len(points) = ", len(points))
    points = filter_points_by_distance_rtree(points, 7)
    print("len(points) = ", len(points))
    cropped_image = four_point_transform_whole_table(image_original, points)
    
    return cropped_image

In [12]:
def profesionally_crop_all_images(path_images):
    files = os.listdir(path_images)

    # Filter the files to keep only the .jpg images
    jpg_files = [file for file in files if file.endswith(".jpg")]

    # Iterate through the .jpg images and open them with OpenCV
    index = 0
    for jpg_file in jpg_files:
        index += 1
        print("cropping the ", jpg_file)
        image_path = os.path.join(path_images, jpg_file)
        image = cv2.imread(image_path)
        #cropped_image = process_image(image)
        #find_green_diamonds(image)
        #find_brown_margins(image)
        cropped_image = profesionally_crop_image(image)
        cv2.imwrite("croppedImagesRest/cropped_again_" + jpg_file, cropped_image)
        

In [13]:
profesionally_crop_all_images(path_regular_task)

cropping the  1_01.jpg
filtering...
len(points) =  102021
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
len(points) =  98917
cropping the  1_02.jpg
filtering...
len(points) =  124290
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
	filtering =  120000
len(points) =  121590
cropping the  1_03.jpg
filtering...
len(points) =  98837
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
len(points) =  96080
cropping the  1_04.jpg
filtering...
len(points) =  138358
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
	filtering =  120000
len(points) =  135575
cropping the  1_05.jpg
filtering...
len(points) =  127878
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
	filtering =  12000

	filtering =  140000
	filtering =  160000
	filtering =  180000
len(points) =  179512
cropping the  2_15.jpg
filtering...
len(points) =  173702
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
	filtering =  120000
	filtering =  140000
	filtering =  160000
len(points) =  171045
cropping the  2_16.jpg
filtering...
len(points) =  126074
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
	filtering =  120000
len(points) =  123612
cropping the  2_17.jpg
filtering...
len(points) =  148121
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
	filtering =  120000
	filtering =  140000
len(points) =  145371
cropping the  2_18.jpg
filtering...
len(points) =  176040
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
	filtering =  80000
	filtering =  100000
	filtering =  120000
	f

	filtering =  0
	filtering =  20000
	filtering =  40000
len(points) =  55872
cropping the  5_07.jpg
filtering...
len(points) =  64346
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
len(points) =  63159
cropping the  5_08.jpg
filtering...
len(points) =  68074
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
len(points) =  66412
cropping the  5_09.jpg
filtering...
len(points) =  53728
	filtering =  0
	filtering =  20000
	filtering =  40000
len(points) =  52730
cropping the  5_10.jpg
filtering...
len(points) =  50773
	filtering =  0
	filtering =  20000
	filtering =  40000
len(points) =  49802
cropping the  5_11.jpg
filtering...
len(points) =  62411
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
len(points) =  61319
cropping the  5_12.jpg
filtering...
len(points) =  66243
	filtering =  0
	filtering =  20000
	filtering =  40000
	filtering =  60000
len(points) =  64455
cropping the  5_13.jpg
filtering...
len

In [14]:
print("the section where I use the already-cropped images and I compute the scores...")

the section where I use the already-cropped images and I compute the scores...


In [5]:
line_1 = [(276, 268), (345, 269), (413, 270), (481, 271), (548, 271), (616, 271), (684, 272), (751, 273), (818, 273), (886, 274), (953, 274), (1021, 274), (1088, 274), (1156, 274), (1225, 273)]
line_2 = [(277, 337), (345, 338), (413, 338), (481, 339), (549, 340), (616, 340), (684, 341), (751, 342), (818, 342), (885, 342), (953, 342), (1021, 343), (1088, 342), (1156, 343), (1224, 343)]
line_3 = [(277, 406), (346, 407), (413, 407), (481, 408), (549, 408), (617, 409), (684, 410), (751, 411), (818, 411), (886, 411), (953, 411), (1021, 411), (1088, 411), (1156, 411), (1224, 411)]
line_4 = [(277, 474), (346, 475), (414, 476), (482, 476), (550, 477), (617, 478), (685, 479), (752, 479), (819, 480), (886, 479), (953, 479), (1021, 479), (1088, 480), (1156, 479), (1224, 480)]
line_5 = [(277, 543), (346, 544), (414, 544), (482, 545), (550, 546), (618, 547), (685, 547), (752, 548), (819, 548), (886, 548), (953, 548), (1021, 548), (1088, 548), (1156, 549), (1224, 549)]
line_6 = [(278, 612), (346, 612), (414, 613), (482, 613), (550, 614), (618, 615), (685, 615), (752, 616), (819, 616), (886, 616), (953, 616), (1021, 616), (1088, 616), (1156, 616), (1225, 617)]
line_7 = [(277, 680), (346, 680), (414, 681), (483, 682), (551, 682), (618, 682), (685, 682), (753, 683), (819, 683), (886, 684), (953, 685), (1021, 685), (1089, 685), (1157, 685), (1225, 685)]

line_8 = [(277, 749), (344, 751), (416, 753), (482, 751), (551, 753), (620, 751), (686, 748), (753, 749), (820, 749), (887, 755), (955, 755),
                     (1022, 754),
                     (1089, 755),
                     (1158, 755),
                     (1225, 758)]

line_9 = [(277, 818), (347, 819), (415, 819), (483, 819), (551, 819), (619, 820), (686, 820), (752, 819), (819, 821), (886, 822), (953, 822), (1021, 822), (1089, 823), (1157, 823), (1225, 823)]
line_10 = [(278, 887), (347, 887), (415, 887), (483, 887), (551, 887), (619, 888), (686, 887), (753, 888), (819, 888), (886, 889), (953, 890), (1021, 890), (1089, 891), (1157, 891), (1225, 892)]
line_11 = [(278, 955), (347, 956), (415, 956), (483, 956), (551, 956), (619, 956), (686, 956), (753, 956), (819, 956), (886, 957), (954, 958), (1021, 958), (1089, 959), (1157, 960), (1226, 960)]
line_12 = [(278, 1024), (347, 1024), (415, 1024), (483, 1024), (551, 1024), (619, 1025), (686, 1025), (753, 1025), (819, 1025), (887, 1026), (954, 1026), (1022, 1027), (1090, 1028), (1157, 1028), (1226, 1029)]
line_13 = [(279, 1093), (347, 1093), (415, 1093), (483, 1093), (551, 1093), (619, 1093), (686, 1093), (753, 1094), (820, 1094), (887, 1094), (954, 1095), (1023, 1096), (1090, 1096), (1158, 1097), (1226, 1098)]
line_14 = [(279, 1162), (347, 1162), (415, 1162), (483, 1162), (551, 1162), (618, 1162), (686, 1162), (753, 1163), (820, 1163), (888, 1163), (955, 1164), (1023, 1164), (1090, 1165), (1158, 1166), (1226, 1166)]
line_15 = [(279, 1231), (347, 1230), (415, 1230), (483, 1231), (551, 1231), (618, 1231), (686, 1231), (753, 1231), (820, 1232), (888, 1232), (955, 1232), (1023, 1233), (1091, 1234), (1159, 1234), (1227, 1235)]


In [6]:
black_pixels = [(36, 37, 41), (35, 38, 47), (33, 39, 51), (66, 82, 107), (24, 26, 25), (23, 29, 29), (51, 66, 73), (48, 65, 75), (68, 72, 84), (36, 37, 39), (0, 1, 0), (22, 27, 31), (32, 46, 59), (6, 6, 6), (27, 29, 28), (49, 63, 76), (2, 6, 7), (4, 6, 5), (27, 29, 28), (17, 36, 40)]

In [7]:
white_pixels_from_pieces = [(204, 222, 200), (202, 224, 203), (203, 220, 202), (196, 213, 194), (206, 224, 198), (210, 225, 204), (204, 222, 198), (202, 220, 198), (199, 217, 195), (199, 217, 195)]

In [8]:
matrix_centers = [line_1,
                 line_2, 
                 line_3,
                 line_4,
                 line_5,
                 line_6,
                 line_7,
                 line_8,
                 line_9,
                 line_10,
                 line_11,
                 line_12,
                 line_13,
                 line_14,
                 line_15
                 ]

In [9]:
all_centers_list = []

for row in matrix_centers:
    print("len(row) = ", len(row))
    all_centers_list += row

len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15
len(row) =  15


In [10]:
all_centers_list

[(276, 268),
 (345, 269),
 (413, 270),
 (481, 271),
 (548, 271),
 (616, 271),
 (684, 272),
 (751, 273),
 (818, 273),
 (886, 274),
 (953, 274),
 (1021, 274),
 (1088, 274),
 (1156, 274),
 (1225, 273),
 (277, 337),
 (345, 338),
 (413, 338),
 (481, 339),
 (549, 340),
 (616, 340),
 (684, 341),
 (751, 342),
 (818, 342),
 (885, 342),
 (953, 342),
 (1021, 343),
 (1088, 342),
 (1156, 343),
 (1224, 343),
 (277, 406),
 (346, 407),
 (413, 407),
 (481, 408),
 (549, 408),
 (617, 409),
 (684, 410),
 (751, 411),
 (818, 411),
 (886, 411),
 (953, 411),
 (1021, 411),
 (1088, 411),
 (1156, 411),
 (1224, 411),
 (277, 474),
 (346, 475),
 (414, 476),
 (482, 476),
 (550, 477),
 (617, 478),
 (685, 479),
 (752, 479),
 (819, 480),
 (886, 479),
 (953, 479),
 (1021, 479),
 (1088, 480),
 (1156, 479),
 (1224, 480),
 (277, 543),
 (346, 544),
 (414, 544),
 (482, 545),
 (550, 546),
 (618, 547),
 (685, 547),
 (752, 548),
 (819, 548),
 (886, 548),
 (953, 548),
 (1021, 548),
 (1088, 548),
 (1156, 549),
 (1224, 549),
 (278

In [11]:
def point_to_coords(point, memo_dict):
    if memo_dict is None:
        memo_dict = {}
    
    if point in memo_dict:
        return memo_dict[point]

    for row in range(15):
        for column in range(15):
            if point == matrix_centers[row][column]:
                coords = (row, column)
                memo_dict[point] = coords
                return coords
                
    memo_dict[point] = None
    t = 7/0
    return None

#important line
memo_dict = {}



def neighbours(point1, point2):
    row1, column1 = point_to_coords(point1, memo_dict)
    row2, column2 = point_to_coords(point2, memo_dict)
    
    if row1==row2 and abs(column1 - column2)==1:
        return True
    if column1==column2 and abs(row1 - row2)==1:
        return True
    return False
def all_neighbours(point):
    dx = [ 0, 0, 1,-1]
    dy = [ 1,-1, 0, 0]
    
    row, column = point_to_coords(point, memo_dict)
    
    neighbours_point = []
    
    for k in range(4):
        row_ngh = row+dx[k]
        column_ngh = column+dy[k]
        
        if row_ngh>=0 and row_ngh<15 and column_ngh>=0 and column_ngh<15:
            neighbours_point.append(matrix_centers[row_ngh][column_ngh])
    
    return neighbours_point  

In [12]:
memo_dict

{}

In [13]:
len(all_centers_list)

225

In [14]:
len(line_8)

15

In [15]:
centers_of_cells = [(277, 749),
                     (344, 751),
                     (416, 753),
                     (482, 751),
                     (551, 753),
                     (620, 751),
                     (686, 748),
                     (753, 749),
                     (820, 749),
                     (887, 755),
                     (955, 755),
                     (1022, 754),
                     (1089, 755),
                     (1158, 755),
                     (1225, 758)]

In [16]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [17]:
all_diffs = [[] for i in range(15)]

In [18]:
#the part that counts the number of black dots
def is_color_close(color1, color2, threshold=17):
    return np.linalg.norm(np.array(color1) - np.array(color2)) <= threshold

def fill_algorithm(img, x, y, window_size, threshold, colors, replacing_color=(0, 255, 0)):
    height, width, _ = img.shape
    visited = set()
    to_visit = deque([(x, y)])
    min_x, max_x = x, x + window_size
    min_y, max_y = y, y + window_size
    count = 0
    while to_visit:
        x, y = to_visit.popleft()
        if (x, y) in visited:
            continue

        visited.add((x, y))
        window = img[y:y + window_size, x:x + window_size]
        avg_color = np.mean(window, axis=(0, 1))

        for color in colors:
            
            if is_color_close(color, avg_color, threshold):
                img[y:y + window_size, x:x + window_size] = replacing_color
                count += 1
                min_x, max_x = min(min_x, x), max(max_x, x)
                min_y, max_y = min(min_y, y), max(max_y, y)
                
                for dx, dy in [(0, window_size), (0, -window_size), (window_size, 0), (-window_size, 0)]:
                    new_x, new_y = x + dx, y + dy
                    if 0 <= new_x < width - window_size and 0 <= new_y < height - window_size:
                        to_visit.append((new_x, new_y))

                break

    return count, min_x, max_x, min_y, max_y

In [19]:
def color_distance_euclidean(color1, color2):
    return np.linalg.norm(np.array(color1) - np.array(color2))

def is_color_very_close(color, color_list, tolerance):
    for target_color in color_list:
        if color_distance_euclidean(color, target_color) <= tolerance:
            return True
    return False

the_letter_color_list = [(12, 29, 10), (29, 40, 26), (38, 41, 34), (40, 39, 37), (43, 41, 42), (31, 34, 27), (5, 17, 7), (17, 31, 18), (29, 35, 21), (43, 47, 46), (47, 48, 42), (46, 45, 41), (47, 48, 43), (45, 44, 39), (43, 46, 39), (18, 28, 19), (48, 48, 40), (25, 44, 40)]

def contains_problematic_letter(image, width=7, height=25, color_list=the_letter_color_list, tolerance=30):
    if image.shape[0] < height or image.shape[1] < width:
        return False

    for y in range(image.shape[0] - height + 1):
        for x in range(image.shape[1] - width + 1):
            rect_colors = image[y:y + height, x:x + width].reshape(-1, 3)
            all_colors_close = all(is_color_very_close(color, color_list, tolerance) for color in rect_colors)

            if all_colors_close:
                return True

    return False


In [20]:
def possibly_contains_problematic_letter(point):
    row, column = point_to_coords(point, memo_dict)
    
    if (row==6 or row==7) and (column>=6 and column<=8):
        return True
    return False

In [21]:
def compare_histograms(imageA, imageB):
    # Compute the histograms for the three color channels
    histA = [cv2.calcHist([imageA], [i], None, [256], [0, 256]) for i in range(3)]
    histB = [cv2.calcHist([imageB], [i], None, [256], [0, 256]) for i in range(3)]

    # Compute the difference between the histograms
    diff = sum([cv2.compareHist(histA[i], histB[i], cv2.HISTCMP_CHISQR) for i in range(3)])
    return diff

def compare_hog(imageA, imageB):
    # Convert the images to grayscale
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    
    # Calculate the HOG features for both images
    hogA = hog(grayA, orientations=9, pixels_per_cell=(5, 5), cells_per_block=(2, 2), transform_sqrt=True)
    hogB = hog(grayB, orientations=9, pixels_per_cell=(5, 5), cells_per_block=(2, 2), transform_sqrt=True)
    
    # Compute the difference between the HOG features
    diff = np.sum((hogA - hogB) ** 2)
    return diff

def compare_black_pixels(imageA, imageB):
    imageA_copy = imageA.copy()
    imageB_copy = imageB.copy()
    
    nb1 = find_black_pixels(imageA_copy)
    nb2 = find_black_pixels(imageB_copy)
    
    if nb1==nb2:
        return 0
    return 1



def most_different_squares(imageA, imageB, points, square_size):
    max_diff = -1
    second_max_diff = -1
    most_different_point = None
    second_most_different_point = None
    
    lin = 0
    col = 0
    
    all_diffs = [[] for i in range(15)]
    print("la comparare, len points =  ", len(points))
    for point in points:
        
        col+=1
        if col==16:
            col = 1
            lin+=1
        
        x, y = point
        x_start = max(x - square_size // 2, 0)
        y_start = max(y - square_size // 2, 0)

        squareA = imageA[y_start:y_start + square_size, x_start:x_start + square_size]
        squareB = imageB[y_start:y_start + square_size, x_start:x_start + square_size]
        
        if possibly_contains_problematic_letter(point):
            print("possibly contains problematic letter. Does it?")
            if contains_problematic_letter(squareB):
                print("yes it contains")
                continue
            else:
                print("no it does not")
                
        
        
        diff = mse(squareA, squareB)#compare_black_pixels(squareA, squareB)##compare_hog(squareA, squareB)#compare_histograms(squareA, squareB)## 
        all_diffs[lin].append(diff)
        if diff > max_diff:
            second_max_diff = max_diff
            second_most_different_point = most_different_point

            max_diff = diff
            most_different_point = point
        elif diff > second_max_diff:
            second_max_diff = diff
            second_most_different_point = point
    print("max diff este ", max_diff)
    print("second max diff este ", second_max_diff)
    
    #print("all diffs este:", all_diffs)
    
    #cv.imshow('squareA', most_different_squareA)
    #cv.waitKey(0)
    #cv.destroyAllWindows()
    
    #cv.imshow('squareB', most_different_squareB)
    #cv.waitKey(0)
    #cv.destroyAllWindows()
    
    #cv.imshow('second_most_different_squareA', second_most_different_squareA)
    #cv.waitKey(0)
    #cv.destroyAllWindows()
    
    #cv.imshow('second_most_different_squareB', second_most_different_squareB)
    #cv.waitKey(0)
    #cv.destroyAllWindows()
    print("returning different squares: ", most_different_point, second_most_different_point)
    return most_different_point, second_most_different_point


In [22]:
def find_most_different_squares(imageA, imageB, points, square_size=60):
    imageA_copy = imageA.copy()
    imageB_copy = imageB.copy()
    
    if len(points)<5:
        print("the received points are: ", points)
    
    most_different_point, second_most_different_point = most_different_squares(imageA_copy, imageB_copy, points, square_size)
    
    x_startA = max(most_different_point[0] - square_size // 2, 0)
    y_startA = max(most_different_point[1] - square_size // 2, 0)
    x_startB = max(second_most_different_point[0] - square_size // 2, 0)
    y_startB = max(second_most_different_point[1] - square_size // 2, 0)

    
    return x_startA, y_startA, x_startB, y_startB, square_size, most_different_point, second_most_different_point


In [23]:
def draw_square(image, x_start, y_start, square_size, color):
    cv2.rectangle(image, (x_start, y_start), (x_start + square_size, y_start + square_size), color, 2)
    
    return image


In [24]:
def find_black_pixels(img):
    height, width, _ = img.shape
    window_size = 2
    threshold = 17
    colors = black_pixels
    
    list_green_diamonds = []
    
    for y in range(0, height - window_size + 1, 1):
        #if y<10 or y%400==0:
        #    print("y = ", y)
        for x in range(0, width - window_size + 1, 1):
            #if (y<10 or y%200==0) and x%1000==0:
            #    print("\tx = ", x)
            window = img[y:y + window_size, x:x + window_size]
            avg_color = np.mean(window, axis=(0, 1))

            for color in colors:
                if is_color_close(color, avg_color, threshold):
                    count, min_x, max_x, min_y, max_y = fill_algorithm(img, x, y, window_size, threshold, colors)
                    smaller_side = min(max_x - min_x, max_y - min_y)
                    bigger_side = max(max_x - min_x, max_y - min_y)
                    raport = 1.0*bigger_side/smaller_side
                    if count >= 13 and (max_y - min_y <20) and (max_x - min_x <20) and (raport<2.4):
                        rect = [min_x, max_x, min_y, max_y]
                        print(f"Filled {count} windows with min_x={min_x}, max_x={max_x}, min_y={min_y}, max_y={max_y}")
                        print("min_x, max_x, min_y, max_y = ", min_x, max_x, min_y, max_y)
                        print("iar raportul a fost ", raport)
                        print("\n\n")
                        list_green_diamonds.append(rect)
                    elif count>1:
                        print("\t not added to dots list. count =  ", count)
                        print("\t also not added to dots list. min_x, max_x, min_y, max_y =  ", min_x, max_x, min_y, max_y)
                        print("\t...iar raportul a fost", raport)
    print("list_green_diamonds.count = ", len(list_green_diamonds))
    
    return list_green_diamonds

In [25]:
def shrink_square_image(img):
    height, width, _ = img.shape
    window_size = 2
    threshold = 35
    colors = white_pixels_from_pieces
    list_green_diamonds = []
    for y in range(0, height - window_size + 1, 1):
        #if y<10 or y%400==0:
        #    print("y = ", y)
        for x in range(0, width - window_size + 1, 1):
            #if (y<10 or y%200==0) and x%1000==0:
            #    print("\tx = ", x)
            window = img[y:y + window_size, x:x + window_size]
            avg_color = np.mean(window, axis=(0, 1))

            for color in colors:
                if is_color_close(color, avg_color, threshold):
                    count, min_x, max_x, min_y, max_y = fill_algorithm(img, x, y, window_size, threshold, colors, (255, 0, 0))
                    if count > 3 and (max_y - min_y <20) and (max_x - min_x <20):
                        rect = [min_x, max_x, min_y, max_y]
                        #print(f"Filled {count} windows with min_x={min_x}, max_x={max_x}, min_y={min_y}, max_y={max_y}")
                        #print("min_x, max_x, min_y, max_y = ", min_x, max_x, min_y, max_y)
                        #print("\n\n")
                        list_green_diamonds.append(rect)
    #print("list_green_diamonds.count = ", len(list_green_diamonds))
    
    return list_green_diamonds

In [26]:
def nth_uppercase_letter(n):
    return chr(ord('A') + n)

def point_to_code(point):
    row, column = point_to_coords(point, memo_dict)
    
    row+=1
    letter = nth_uppercase_letter(column)
    
    return str(row) + letter

In [27]:
point_to_code(matrix_centers[0][0])

'1A'

In [28]:
len(all_centers_list)

225

In [29]:
def replace_jpg_with_txt(file_name):
    if file_name.endswith('.jpg'):
        return file_name[:-4] + '.txt'
    else:
        raise ValueError("The input string does not end with .jpg")

In [30]:
def create_lines_dots_txt(most_different_point, second_most_different_point, count_black_dots1, count_black_dots2):
    row1, column1 = point_to_coords(most_different_point, memo_dict)
    row2, column2 = point_to_coords(second_most_different_point, memo_dict)
    
    if row1==row2:
        if column1<column2:#normal, don't sort
            return point_to_code(most_different_point), point_to_code(second_most_different_point), count_black_dots1, count_black_dots2
        else:#not normal, sort
            return point_to_code(second_most_different_point), point_to_code(most_different_point), count_black_dots2, count_black_dots1
    
    #else, the columns are equal
    if row1<row2:#normal, don't sort
            return point_to_code(most_different_point), point_to_code(second_most_different_point), count_black_dots1, count_black_dots2
    #else, not normal, sort
    return point_to_code(second_most_different_point), point_to_code(most_different_point), count_black_dots2, count_black_dots1    
        

In [31]:
marginal_points_for_score = [-1,1,2,3,4,5,6,0,2,5,3,4,6,2,2,0,3,5,4,1,6,2,4,5,5,0,6,3,4,2,0,1,5,1,3,4,4,4,5,0,6,3,5,4,1,3,2,0,0,1,1,2,3,6,3,5,2,1,0,6,6,5,2,1,2,5,0,3,3,5,0,6,1,4,0,6,3,5,1,4,2,6,2,3,1,6,5,6,2,1,4,0,1,6,4,4,1,6,6,3,0]

In [32]:
diamonds_values = [
    [5,0,0,4,0,0,0,3,0,0,0,4,0,0,5],
    [0,0,3,0,0,4,0,0,0,4,0,0,3,0,0],
    [0,3,0,0,2,0,0,0,0,0,2,0,0,3,0],
    [4,0,0,3,0,2,0,0,0,2,0,3,0,0,4],
    [0,0,2,0,1,0,1,0,1,0,1,0,2,0,0],
    [0,4,0,2,0,1,0,0,0,1,0,2,0,4,0],
    [0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
    
    [3,0,0,0,0,0,0,0,0,0,0,0,0,0,3],
    
    [0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
    [0,4,0,2,0,1,0,0,0,1,0,2,0,4,0],
    [0,0,2,0,1,0,1,0,1,0,1,0,2,0,0],
    [4,0,0,3,0,2,0,0,0,2,0,3,0,0,4],
    [0,3,0,0,2,0,0,0,0,0,2,0,0,3,0],
    [0,0,3,0,0,4,0,0,0,4,0,0,3,0,0],
    [5,0,0,4,0,0,0,3,0,0,0,4,0,0,5]
]

In [33]:
len(diamonds_values)

15

In [34]:
len(marginal_points_for_score)

101

In [35]:
def compute_added_score(most_different_point, second_most_different_point, count_black_dots1, count_black_dots2):
    row1, column1 = point_to_coords(most_different_point, memo_dict)
    row2, column2 = point_to_coords(second_most_different_point, memo_dict)
    
    added_score = diamonds_values[row1][column1] + diamonds_values[row2][column2]
    
    if count_black_dots1 == count_black_dots2:
        return 2*added_score
    
    return added_score

In [37]:
def run_project(path_images,path_txt_files_overall, path_starting_image):
    files = os.listdir(path_images)
    files_txt_final = os.listdir(path_txt_files_overall)

    # Filter the files to keep only the .jpg images
    jpg_files = [file for file in files if file.endswith(".jpg")]
    txt_files = [file for file in files_txt_final if file.endswith(".txt")]
    
    
    index = 0
    previous_image = cv2.imread(path_starting_image)
    visited_squares = []
    index_txt_file = 0
    originally_previous_image = previous_image.copy()
    
    current_data_from_txt = []
    
    
    index_txt_file = 0
    
    file_path = os.path.join(path_txt_files_overall, txt_files[index_txt_file])
    print("file_path is ", file_path)
    with open(file_path, 'r') as file:
        for line in file:
            filename, player = line.strip().split(' ')
            current_data_from_txt.append((filename, player))
    
    score_player1 = 0
    score_player2 = 0
    
    for jpg_file in jpg_files:
        index += 1
        print("\n\n\n\n\n\n\n\n\nprocessing ", jpg_file, "...")
        
        image_path = os.path.join(path_images, jpg_file)
        current_image_initial = cv2.imread(image_path)
        current_image = current_image_initial.copy()
        
        
        unused_points = list(set(all_centers_list) - set(visited_squares))
        x_startA, y_startA, x_startB, y_startB, square_size,  most_different_point, second_most_different_point = find_most_different_squares(previous_image, current_image, unused_points, 65)
        
        
        if neighbours(most_different_point, second_most_different_point)==False:
            print("They are not neighbours!!!")
            print("they coords are: ", point_to_coords(most_different_point, memo_dict), " and ", point_to_coords(second_most_different_point, memo_dict))
            unused_points = all_neighbours(most_different_point)
            print("neigbours of first point are", unused_points)
            if len(unused_points)>0:
                x_startB, y_startB, _, _, square_size,  real_second_most_different_point, _ = find_most_different_squares(previous_image, current_image, unused_points, 65)
                second_most_different_point = real_second_most_different_point
                                                              
        visited_squares.append(most_different_point)
        visited_squares.append(second_most_different_point)
        
        
        
        print("at main: ", x_startA, y_startA, x_startB, y_startB, square_size)
        print("visited squares: ", visited_squares)
        
        little_square1 = current_image[y_startA:y_startA + square_size, x_startA:x_startA + square_size]
        little_square2 = current_image[y_startB:y_startB + square_size, x_startB:x_startB + square_size]
        
        
        draw_square(current_image, x_startA, y_startA, square_size, (0, 255, 0))
        draw_square(current_image, x_startB, y_startB, square_size, (0, 0, 255))
        
        
        #shrink_square_image(little_square)
        count_black_dots1 = find_black_pixels(little_square1)
        count_black_dots2 = find_black_pixels(little_square2)
        
        #cv2.imwrite("OnlyTwoComparedImages2/__AllSets_AlmostFinal_" + str(count_black_dots1) + "_sqA_" + jpg_file, little_square1)
        #cv2.imwrite("OnlyTwoComparedImages2/__AllSets_AlmostFinal_" + str(count_black_dots2) + "_sqB_" + jpg_file, little_square2)
        
        
        
        #detectedImage = draw_square(current_image, x_startB, y_startB, square_size)
        
        
        #Compute the scores and the file output
        #Compute the scores and the file output
        #Compute the scores and the file output
        
        #sort the points if needed:
        code1, code2, number1, number2 = create_lines_dots_txt(most_different_point, second_most_different_point, len(count_black_dots1), len(count_black_dots2))
        
        #compute the actual score
        
        #check for bonus first
        #player1
        bonus_score_1 = 0
        bonus_score_2 = 0
        
        if marginal_points_for_score[score_player1] ==  len(count_black_dots1) or marginal_points_for_score[score_player1] ==  len(count_black_dots2):
            score_player1 += 3
            bonus_score_1 = 3
        if marginal_points_for_score[score_player2] ==  len(count_black_dots1) or marginal_points_for_score[score_player2] ==  len(count_black_dots2):
            score_player2 += 3
            bonus_score_2 = 3
        
        
        classically_added_score = compute_added_score(most_different_point, second_most_different_point, len(count_black_dots1), len(count_black_dots2))
        print("current data is ", current_data_from_txt[(index-1)%20])
        if current_data_from_txt[(index-1)%20][1] == "player1":
            score_player1 += classically_added_score
        elif current_data_from_txt[(index-1)%20][1] == "player2":
            score_player2 += classically_added_score
        else:
            t = 7/0
        
        #write it in the file
        file_name_txt = replace_jpg_with_txt(jpg_file)
        
        
        if current_data_from_txt[(index-1)%20][1] == "player1":
            score = classically_added_score + bonus_score_1
        elif current_data_from_txt[(index-1)%20][1] == "player2":
            score = classically_added_score + bonus_score_2
        
        file_content = ""
        file_content += code1 + " " + str(number1) + "\n"
        file_content += code2 + " " + str(number2) + "\n"
        file_content += str(score) + "\n"
        
        
        
        
        
        f = open("TextResults/" + file_name_txt, "w")
        f.write(file_content)
        f.close()
        
        
        
        #Compute the scores and the file output
        #Compute the scores and the file output
        #Compute the scores and the file output
        
        
        
        
        cv2.imwrite("NewMethodDetected/mse_no_repetition_ngh/_finaltest_detected" + "_" + jpg_file, current_image)
        if index%20==0:
            score_player1 = 0
            score_player2 = 0
            previous_image = originally_previous_image.copy()
            visited_squares = []
            index_txt_file+=1
            if index_txt_file < len(txt_files):
                file_path = os.path.join(path_txt_files_overall, txt_files[index_txt_file])
                current_data_from_txt = []
                with open(file_path, 'r') as file:
                    ind_line = 0
                    for line in file:
                        ind_line+=1
                        if ind_line==21:
                            break
                        print("after 20, the line is ", line)
                        filename, player = line.strip().split(' ')
                        current_data_from_txt.append((filename, player))
        else:
            previous_image = current_image_initial.copy()
        #if index==3:
        #break

In [38]:
#path_processed_images = "C:\\Users\\Alienware\\source\\CV-2023-Project1\\readyToCountImages"
#path_general_txt_files = "C:\\Users\\Alienware\\source\\CV-2023-Project1\\GeneralText"

In [39]:
run_project(path_processed_images,path_general_txt_files, path_initial_image)

file_path is  C:\Users\Alienware\source\507_Oanea_Smit\testImages\regular_tasks\1_moves.txt









processing  cropped_again_1_01.jpg ...
la comparare, len points =   225
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
no it does not
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
max diff este  36423.832899408284
second max diff este  35627.68781065089
returning different squares:  (752, 819) (753, 749)
at main:  720 787 721 717 65
visited squares:  [(752, 819), (753, 749)]
Filled 41 windows with min_x=41, max_x=53, min_y=4, max_y=18
min_x, max_x, min_y, max_y =  41 53 4 18
iar raportul a fost  1.1666666666666667



	 not added to dots list. count =   6
	 also not added to dots list. min_x, max_x, min_y, max_y =   62 64 1

	 not added to dots list. count =   2
	 also not added to dots list. min_x, max_x, min_y, max_y =   29 31 36 38
	...iar raportul a fost 1.0
Filled 29 windows with min_x=2, max_x=12, min_y=45, max_y=57
min_x, max_x, min_y, max_y =  2 12 45 57
iar raportul a fost  1.2



Filled 31 windows with min_x=39, max_x=49, min_y=45, max_y=59
min_x, max_x, min_y, max_y =  39 49 45 59
iar raportul a fost  1.4



	 not added to dots list. count =   2
	 also not added to dots list. min_x, max_x, min_y, max_y =   51 53 52 54
	...iar raportul a fost 1.0
list_green_diamonds.count =  5
Filled 32 windows with min_x=39, max_x=51, min_y=7, max_y=21
min_x, max_x, min_y, max_y =  39 51 7 21
iar raportul a fost  1.1666666666666667



Filled 21 windows with min_x=3, max_x=9, min_y=8, max_y=20
min_x, max_x, min_y, max_y =  3 9 8 20
iar raportul a fost  2.0



	 not added to dots list. count =   3
	 also not added to dots list. min_x, max_x, min_y, max_y =   62 64 9 13
	...iar raportul a fost 2.0
	 not added to do

	 not added to dots list. count =   5
	 also not added to dots list. min_x, max_x, min_y, max_y =   62 64 36 44
	...iar raportul a fost 4.0
Filled 30 windows with min_x=3, max_x=13, min_y=42, max_y=54
min_x, max_x, min_y, max_y =  3 13 42 54
iar raportul a fost  1.2



	 not added to dots list. count =   5
	 also not added to dots list. min_x, max_x, min_y, max_y =   62 64 53 61
	...iar raportul a fost 4.0
	 not added to dots list. count =   5
	 also not added to dots list. min_x, max_x, min_y, max_y =   4 12 62 64
	...iar raportul a fost 4.0
	 not added to dots list. count =   5
	 also not added to dots list. min_x, max_x, min_y, max_y =   20 28 62 64
	...iar raportul a fost 4.0
list_green_diamonds.count =  2
	 not added to dots list. count =   29
	 also not added to dots list. min_x, max_x, min_y, max_y =   59 63 2 52
	...iar raportul a fost 12.5
Filled 28 windows with min_x=21, max_x=33, min_y=22, max_y=34
min_x, max_x, min_y, max_y =  21 33 22 34
iar raportul a fost  1.0



	 not a

	 not added to dots list. count =   17
	 also not added to dots list. min_x, max_x, min_y, max_y =   2 4 25 57
	...iar raportul a fost 16.0
Filled 35 windows with min_x=13, max_x=25, min_y=44, max_y=58
min_x, max_x, min_y, max_y =  13 25 44 58
iar raportul a fost  1.1666666666666667



Filled 41 windows with min_x=32, max_x=44, min_y=45, max_y=59
min_x, max_x, min_y, max_y =  32 44 45 59
iar raportul a fost  1.1666666666666667



Filled 32 windows with min_x=52, max_x=62, min_y=45, max_y=57
min_x, max_x, min_y, max_y =  52 62 45 57
iar raportul a fost  1.2



	 not added to dots list. count =   2
	 also not added to dots list. min_x, max_x, min_y, max_y =   3 5 59 61
	...iar raportul a fost 1.0
list_green_diamonds.count =  6
	 not added to dots list. count =   55
	 also not added to dots list. min_x, max_x, min_y, max_y =   3 7 6 58
	...iar raportul a fost 13.0
Filled 36 windows with min_x=12, max_x=24, min_y=6, max_y=20
min_x, max_x, min_y, max_y =  12 24 6 20
iar raportul a fost  1.1

	 not added to dots list. count =   3
	 also not added to dots list. min_x, max_x, min_y, max_y =   58 60 57 59
	...iar raportul a fost 1.0
list_green_diamonds.count =  6
current data is  ('1_13.jpg', 'player2')









processing  cropped_again_1_14.jpg ...
la comparare, len points =   199
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
max diff este  35219.4475739645
second max diff este  31382.29775147929
returning different squares:  (347, 887) (347, 819)
at main:  315 855 315 787 65
visited squares:  [(752, 819), (753, 749), (753, 683), (752, 616), (685, 615), (685, 547), (819, 616), (886, 616), (685, 479), (617, 478), (616, 340), (617, 409), (550, 477), (482, 476), (482, 545), (482, 613), (953, 616), (1021, 616), (1088, 616), (1156, 616), (346, 544), (414, 544), (346, 612), (346

...

Filled 40 windows with min_x=8, max_x=22, min_y=40, max_y=54
min_x, max_x, min_y, max_y =  8 22 40 54
iar raportul a fost  1.0



Filled 48 windows with min_x=48, max_x=62, min_y=41, max_y=55
min_x, max_x, min_y, max_y =  48 62 41 55
iar raportul a fost  1.0



	 not added to dots list. count =   2
	 also not added to dots list. min_x, max_x, min_y, max_y =   21 23 48 50
	...iar raportul a fost 1.0
list_green_diamonds.count =  4
current data is  ('5_04.jpg', 'player2')









processing  cropped_again_5_05.jpg ...
la comparare, len points =   217
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
max diff este  47166.712426035505
second max diff este  39733.692307692305
returning different squares:  (1088, 616) (1156, 616)
at main:  1056 584 1124 584 65
visited squares:  [(752, 819), (7

list_green_diamonds.count =  1
Filled 20 windows with min_x=16, max_x=26, min_y=2, max_y=12
min_x, max_x, min_y, max_y =  16 26 2 12
iar raportul a fost  1.0



Filled 25 windows with min_x=35, max_x=47, min_y=2, max_y=12
min_x, max_x, min_y, max_y =  35 47 2 12
iar raportul a fost  1.2



Filled 24 windows with min_x=53, max_x=61, min_y=2, max_y=12
min_x, max_x, min_y, max_y =  53 61 2 12
iar raportul a fost  1.25



Filled 30 windows with min_x=15, max_x=27, min_y=37, max_y=49
min_x, max_x, min_y, max_y =  15 27 37 49
iar raportul a fost  1.0



Filled 32 windows with min_x=34, max_x=46, min_y=38, max_y=50
min_x, max_x, min_y, max_y =  34 46 38 50
iar raportul a fost  1.0



Filled 26 windows with min_x=53, max_x=61, min_y=38, max_y=50
min_x, max_x, min_y, max_y =  53 61 38 50
iar raportul a fost  1.5



	 not added to dots list. count =   3
	 also not added to dots list. min_x, max_x, min_y, max_y =   58 62 52 54
	...iar raportul a fost 2.0
list_green_diamonds.count =  6
current dat

Filled 40 windows with min_x=2, max_x=14, min_y=40, max_y=54
min_x, max_x, min_y, max_y =  2 14 40 54
iar raportul a fost  1.1666666666666667



Filled 38 windows with min_x=21, max_x=33, min_y=40, max_y=54
min_x, max_x, min_y, max_y =  21 33 40 54
iar raportul a fost  1.1666666666666667



Filled 36 windows with min_x=41, max_x=53, min_y=40, max_y=54
min_x, max_x, min_y, max_y =  41 53 40 54
iar raportul a fost  1.1666666666666667



	 not added to dots list. count =   2
	 also not added to dots list. min_x, max_x, min_y, max_y =   34 36 48 50
	...iar raportul a fost 1.0
	 not added to dots list. count =   6
	 also not added to dots list. min_x, max_x, min_y, max_y =   58 64 55 61
	...iar raportul a fost 1.0
list_green_diamonds.count =  6
current data is  ('5_11.jpg', 'player2')









processing  cropped_again_5_12.jpg ...
la comparare, len points =   203
possibly contains problematic letter. Does it?
no it does not
possibly contains problematic letter. Does it?
no it does not
poss

Filled 36 windows with min_x=24, max_x=36, min_y=22, max_y=36
min_x, max_x, min_y, max_y =  24 36 22 36
iar raportul a fost  1.1666666666666667



Filled 34 windows with min_x=5, max_x=17, min_y=41, max_y=55
min_x, max_x, min_y, max_y =  5 17 41 55
iar raportul a fost  1.1666666666666667



	 not added to dots list. count =   3
	 also not added to dots list. min_x, max_x, min_y, max_y =   18 20 47 51
	...iar raportul a fost 2.0
list_green_diamonds.count =  3
current data is  ('5_15.jpg', 'player1')









processing  cropped_again_5_16.jpg ...
la comparare, len points =   195
possibly contains problematic letter. Does it?
yes it contains
possibly contains problematic letter. Does it?
yes it contains
max diff este  41845.88142011834
second max diff este  41419.914792899406
returning different squares:  (1021, 958) (1021, 890)
at main:  989 926 989 858 65
visited squares:  [(752, 819), (753, 749), (820, 749), (887, 755), (955, 755), (953, 685), (953, 616), (1021, 616), (1088, 616), (11

no it does not
possibly contains problematic letter. Does it?
no it does not
max diff este  53382.64023668639
second max diff este  49002.77159763314
returning different squares:  (1088, 411) (1156, 411)
at main:  1056 379 1124 379 65
visited squares:  [(752, 819), (753, 749), (820, 749), (887, 755), (955, 755), (953, 685), (953, 616), (1021, 616), (1088, 616), (1156, 616), (1021, 479), (1021, 548), (886, 616), (819, 616), (1021, 411), (953, 411), (1021, 274), (1021, 343), (885, 342), (886, 411), (819, 480), (886, 479), (685, 682), (686, 748), (1224, 411), (1224, 480), (1224, 343), (1225, 273), (953, 822), (953, 890), (1021, 958), (1021, 890), (1022, 1027), (1023, 1096), (1023, 1233), (1023, 1164), (1088, 411), (1156, 411)]
	 not added to dots list. count =   171
	 also not added to dots list. min_x, max_x, min_y, max_y =   2 22 2 62
	...iar raportul a fost 3.0
	 not added to dots list. count =   3
	 also not added to dots list. min_x, max_x, min_y, max_y =   38 40 4 8
	...iar raportul

In [49]:
def rename_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.startswith("cropped_again_") and file_name.endswith(".txt"):
            new_file_name = file_name.replace("cropped_again_", "", 1)
            old_file_path = os.path.join(folder_path, file_name)
            new_file_path = os.path.join(folder_path, new_file_name)
            os.rename(old_file_path, new_file_path)

folder_path = "C:\\Users\\Alienware\\source\\507_Oanea_Smit\\TextResults"  # Replace this with the path to your folder
rename_files_in_folder(folder_path)

In [72]:
print("Now you can stop running, the following is only for comparing to the ground-truth on my local machine.")

Now you can stop running, the following is only for comparing to the ground-truth on my local machine.


In [6]:
folder1 = "C:\\Users\\Alienware\\source\\507_Oanea_Smit\\TextResults"
folder2 = "C:\\Users\\Alienware\\source\\507_Oanea_Smit\\GroundTruthOficial"

def compare_files(file1, file2):
    with open(file1, "r") as f1, open(file2, "r") as f2:
        line1_f1 = f1.readline().strip()
        line2_f1 = f1.readline().strip()
        line3_f1 = f1.readline().strip()

        line1_f2 = f2.readline().strip()
        line2_f2 = f2.readline().strip()
        line3_f2 = f2.readline().strip()

        return line1_f1 == line1_f2 and line2_f1 == line2_f2 and line3_f1 == line3_f2

folder1_files = sorted(os.listdir(folder1))
folder2_files = sorted(os.listdir(folder2))
count_different = 0
for i in range(100):
    file1 = os.path.join(folder1, folder1_files[i])
    file2 = os.path.join(folder2, folder2_files[i])

    if compare_files(file1, file2):
        print(f"File {i + 1}: First three lines are identical.")
    else:
        count_different+=1
        print(f"\tFile {i + 1}: First three lines are different.")
print("\n\ncount_different = ", count_different)

File 1: First three lines are identical.
File 2: First three lines are identical.
File 3: First three lines are identical.
File 4: First three lines are identical.
	File 5: First three lines are different.
File 6: First three lines are identical.
File 7: First three lines are identical.
File 8: First three lines are identical.
File 9: First three lines are identical.
File 10: First three lines are identical.
File 11: First three lines are identical.
File 12: First three lines are identical.
File 13: First three lines are identical.
	File 14: First three lines are different.
File 15: First three lines are identical.
	File 16: First three lines are different.
	File 17: First three lines are different.
File 18: First three lines are identical.
	File 19: First three lines are different.
File 20: First three lines are identical.
File 21: First three lines are identical.
File 22: First three lines are identical.
File 23: First three lines are identical.
File 24: First three lines are identic

In [46]:
len(memo_dict)

225

In [47]:
15*15


225

In [120]:
image_07 = cv2.imread(path_processed_images + "\\detected_cropped_again_3_07.jpg")
image_08 = cv2.imread(path_processed_images + "\\detected_cropped_again_3_08.jpg")

In [ ]:
 find_most_different_squares(image_07, image_08, unused_points, 65)

In [346]:
current_image_initial = cv2.imread(image_path)
        current_image = current_image_initial.copy()
        
        
        unused_points = list(set(all_centers_list) - set(visited_squares))
        x_startA, y_startA, x_startB, y_startB, square_size,  most_different_point, second_most_different_point = find_most_different_squares(previous_image, current_image, unused_points, 65)
        
        
        if neighbours(most_different_point, second_most_different_point)==False:
            print("They are not neighbours!!!")
            unused_points = all_neighbours(most_different_point)
            print("neigbours are", unused_points)
            if len(unused_points)>0:
                x_startB, y_startB, _, _, square_size,  real_second_most_different_point, _ = find_most_different_squares(previous_image, current_image, unused_points, 65)
                second_most_different_point = real_second_most_different_point
                                                              
        visited_squares.append(most_different_point)
        visited_squares.append(second_most_different_point)
        
        
        
        print("at main: ", x_startA, y_startA, x_startB, y_startB, square_size)
        print("visited squares: ", visited_squares)
        
        #little_square1 = current_image[y_startA:y_startA + square_size, x_startA:x_startA + square_size]
        #little_square2 = current_image[y_startB:y_startB + square_size, x_startB:x_startB + square_size]
        
        
        draw_square(current_image, x_startA, y_startA, square_size, (0, 255, 0))
        draw_square(current_image, x_startB, y_startB, square_size, (0, 0, 255))
        
        
        #shrink_square_image(little_square)
        count_black_dots1 = 10#find_black_pixels(little_square1)
        count_black_dots2 = 10#find_black_pixels(little_square2)
        
        #cv2.imwrite("OnlyTwoComparedImages2/__AllSets_AlmostFinal_" + str(count_black_dots1) + "_sqA_" + jpg_file, little_square1)
        #cv2.imwrite("OnlyTwoComparedImages2/__AllSets_AlmostFinal_" + str(count_black_dots2) + "_sqB_" + jpg_file, little_square2)
        
        
        
        #detectedImage = draw_square(current_image, x_startB, y_startB, square_size)
        
        
        
        cv2.imwrite("NewMethodDetected/mse_no_repetition_ngh/detected" + "_" + jpg_file, current_image)
        if index%20==0:
            previous_image = originally_previous_image.copy()
            visited_squares = []
        else:
            previous_image = current_image_initial.copy()

[[9.490126960344721,
  12.923272131367177,
  11.792696050154884,
  13.587608256556882,
  26.561434045951458,
  15.474877287708175,
  13.45681372299384,
  16.446760411856552,
  12.361106663710883,
  12.871580399994695,
  11.225479501078702,
  14.712132103367125,
  12.503383237029087,
  13.197304453903216,
  10.117746604823257],
 [9.839944534614933,
  9.920666232964589,
  19.21527355234381,
  14.43425251892375,
  11.578373062872725,
  12.648310422408438,
  12.100449413503812,
  13.1461138522018,
  11.334250756125698,
  11.58545605228882,
  11.026578038636465,
  11.998403327643294,
  12.647905876041193,
  10.317751894787545,
  10.518396877086563],
 [10.565850982904962,
  11.570379739793642,
  11.34777365685908,
  10.876513017171444,
  19.231842040003272,
  12.503659497973558,
  9.48718867225569,
  14.938247487318492,
  23.27538548162622,
  10.366277155471904,
  14.68148800804804,
  11.346399335442705,
  10.711018408712206,
  11.209845697464269,
  12.527746208239217],
 [10.892450993797608,

In [284]:
#prepare for ground truth
def delete_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or "move" in filename:
            file_path = os.path.join(folder_path, filename)
            os.remove(file_path)
            print(f"Deleted file: {filename}")

folder_path = "readyForGroundTruth"
delete_files(folder_path)

Deleted file: 1_01.jpg
Deleted file: 1_02.jpg
Deleted file: 1_03.jpg
Deleted file: 1_04.jpg
Deleted file: 1_05.jpg
Deleted file: 1_06.jpg
Deleted file: 1_07.jpg
Deleted file: 1_08.jpg
Deleted file: 1_09.jpg
Deleted file: 1_10.jpg
Deleted file: 1_11.jpg
Deleted file: 1_12.jpg
Deleted file: 1_13.jpg
Deleted file: 1_14.jpg
Deleted file: 1_15.jpg
Deleted file: 1_16.jpg
Deleted file: 1_17.jpg
Deleted file: 1_18.jpg
Deleted file: 1_19.jpg
Deleted file: 1_20.jpg
Deleted file: 1_moves.txt
Deleted file: 2_01.jpg
Deleted file: 2_02.jpg
Deleted file: 2_03.jpg
Deleted file: 2_04.jpg
Deleted file: 2_05.jpg
Deleted file: 2_06.jpg
Deleted file: 2_07.jpg
Deleted file: 2_08.jpg
Deleted file: 2_09.jpg
Deleted file: 2_10.jpg
Deleted file: 2_11.jpg
Deleted file: 2_12.jpg
Deleted file: 2_13.jpg
Deleted file: 2_14.jpg
Deleted file: 2_15.jpg
Deleted file: 2_16.jpg
Deleted file: 2_17.jpg
Deleted file: 2_18.jpg
Deleted file: 2_19.jpg
Deleted file: 2_20.jpg
Deleted file: 2_moves.txt
Deleted file: 3_01.jpg
Delet

In [183]:
import cv2 as cv

In [ ]:
print("in this section I select various points")

In [2]:
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x, y), 3, (255, 0, 0), -1)
        points.append((x, y))
        print(points)
        if len(points) == 15:
            cv2.destroyAllWindows()

image_path = path_regular_task + "\\_start.jpg"
img = cv2.imread(image_path)
points = []

cv2.namedWindow('image')
cv2.setMouseCallback('image', click_event)

while len(points) < 1:
    cv2.imshow('image', img)
    cv2.waitKey(1)

print("Selected points:", points)

[(653, 650)]
Selected points: [(653, 650)]


In [ ]:
print("centers selected")

In [ ]:
#better version:
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        temp_points.append((x, y))
        if len(temp_points) % 2 == 0:
            mid_x = (temp_points[-2][0] + temp_points[-1][0]) // 2
            mid_y = (temp_points[-2][1] + temp_points[-1][1]) // 2
            cv2.circle(img, (mid_x, mid_y), 2, (255, 0, 0), -1)
            points.append((mid_x, mid_y))
        if len(points) == 15:
            cv2.destroyAllWindows()

image_path = path_regular_task + "\\_start.jpg"
img = cv2.imread(image_path)
points = []
temp_points = []

cv2.namedWindow('image')
cv2.setMouseCallback('image', click_event)

while len(points) < 15:
    cv2.imshow('image', img)
    cv2.waitKey(1)

print("Midpoints of selected segments:", points)
 

In [ ]:
print("len(points) = ", len(points))
print("points is ", points)

In [ ]:
print("len(points) = ", len(points))
print("points is ", points)

In [ ]:
line1 = []

In [ ]:
points = [(277, 749), (344, 751), (416, 753), (482, 751), (551, 753), (620, 751), (686, 748), (753, 749), (820, 749), (887, 755), (955, 755), (1022, 754), (1089, 755), (1158, 755), (1225, 758)]

In [ ]:
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x, y), 3, (255, 0, 0), -1)
        points.append((x, y))
        print(points)
        if len(points) == 15:
            cv2.destroyAllWindows()

image_path = path_regular_task + "\\_start.jpg"
img = cv2.imread(image_path)
points = []

cv2.namedWindow('image')
cv2.setMouseCallback('image', click_event)

while len(points) < 2:
    cv2.imshow('image', img)
    cv2.waitKey(1)

print("Selected points:", points)

In [303]:
with open("input_file_packages.txt", "r") as infile:
    with open("output_file.txt", "w") as outfile:
        for line in infile:
            words = line.strip().split()
            first_two_elements = words[:2]
            output = words[0] + "==" + words[1] + "\n"
            outfile.write(output)